In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Data

In [19]:
datapath = "dataset/"

df = pd.read_csv(datapath + 'data.csv')

In [20]:
df.head()

,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hfouls,afouls,hsaves,asaves,label
0,24,76,2,4,14,17,11,13,2,1,2
1,80,20,3,1,5,2,1,3,0,1,1
2,57,43,4,2,12,11,10,12,1,2,1
3,47,53,5,3,13,11,7,10,1,3,0
4,72,28,5,4,21,6,8,13,4,5,0


In [21]:
df.shape

(1272, 11)

# Nueral Net From Scratch

### Selected Data to Train

In [22]:
sample_df = df.sample(n=100, random_state=42)
sample_df.head()

,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hfouls,afouls,hsaves,asaves,label
208,50,50,7,0,11,2,7,7,0,3,1
966,45,55,1,3,9,10,20,18,3,1,0
714,37,63,2,1,8,10,13,10,1,1,1
1192,64,36,15,3,20,6,15,13,1,7,0
584,28,72,0,5,3,22,12,4,3,0,2


In [23]:
train, test = np.split(sample_df.sample(frac=1, random_state=42), [int(0.8 * len(sample_df))])

train.shape, test.shape

c:\Users\Public\Anaconda\envs\dsde-cp\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


((80, 11), (20, 11))

In [24]:
X = train.drop(columns=['label'])
y = train['label']

X_test = test.drop(columns=['label'])
y_test = test['label']

X.shape, y.shape

((80, 10), (80,))

## init weights

In [25]:
W = np.random.randn(10, 3)
W

array([[-0.02988452,  0.73645279, -1.07051631],
       [ 1.7335747 , -0.74405578, -1.52734829],
       [-0.15952392, -1.39690882,  0.96349871],
       [ 0.34299778, -0.09716324, -0.58550628],
       [-0.55044561,  1.28317046, -0.43439464],
       [ 1.88825096,  0.14621627,  1.72517665],
       [ 0.65339162,  0.44311987, -0.58247862],
       [-2.13753405, -1.27401001,  0.47235221],
       [ 1.19003935,  1.22050342, -0.41457721],
       [-0.61729983, -0.87333001, -1.05037774]])

In [26]:
W0 = np.random.randn(1, 3)
W0

array([[0.27635834, 0.42104474, 2.61168053]])

## Forward Pass

In [27]:
# Forward pass
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / exp_z.sum(axis=1, keepdims=True)

## Original Evaluation

In [28]:
logits = (X @ W + W0).to_numpy()  # Convert to NumPy array to avoid pandas-specific issues
probs = softmax(logits)          # Predicted class probabilities
preds = np.argmax(probs, axis=1) # Class predictions (0, 1, or 2)

In [29]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [30]:
accuracy = accuracy_score(y, preds)

print(f"Original Accuracy: {accuracy:.2f}")

Original Accuracy: 0.28


# Training

### Loss Fuction

In [31]:
def cross_entropy_loss(probs, y_true):
    n = y_true.shape[0]
    log_probs = -np.log(probs[range(n), y_true])
    return np.sum(log_probs) / n

### One-hot encoder
class 0 -> [1, 0, 0]

class 1 -> [0, 1, 0]

class 2 -> [0, 0, 1]


In [32]:
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

In [33]:
learning_rate = 0.1
epochs = 1000
num_classes = 3
n_samples, n_features = X.shape

for epoch in range(epochs):
    # Forward
    logits = np.dot(X, W) + W0
    probs = softmax(logits)
    loss = cross_entropy_loss(probs, y)

    # Back Propagate
    y_onehot = one_hot(y, num_classes)
    dlogits = (probs - y_onehot) / n_samples
    dW = np.dot(X.T, dlogits)
    dw0 = np.sum(dlogits, axis=0, keepdims=True)

    # Update Weights
    W -= learning_rate * dW
    W0 -= learning_rate * dw0

    # Log
    if epoch % 100 == 0:
        predictions = np.argmax(probs, axis=1)
        acc = np.mean(predictions == y) 
        eval_logits = np.dot(X_test, W) + dw0
        eval_probs = softmax(eval_logits)
        eval_preds = np.argmax(eval_probs, axis=1)
        eval_accuracy = accuracy_score(y_test, eval_preds)
        print(f"Test Accuracy: {accuracy:.2f}")
        print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {acc:.2f}, Eval Accuracy: {eval_accuracy:.2f}")

Test Accuracy: 0.28
Epoch 0, Loss: 99.7430, Accuracy: 0.28, Eval Accuracy: 0.60
Test Accuracy: 0.28
Epoch 100, Loss: 19.1487, Accuracy: 0.56, Eval Accuracy: 0.45
Test Accuracy: 0.28
Epoch 200, Loss: 48.6657, Accuracy: 0.39, Eval Accuracy: 0.55
Test Accuracy: 0.28
Epoch 300, Loss: 58.0055, Accuracy: 0.59, Eval Accuracy: 0.55
Test Accuracy: 0.28
Epoch 400, Loss: 49.1947, Accuracy: 0.60, Eval Accuracy: 0.40
Test Accuracy: 0.28
Epoch 500, Loss: 48.4023, Accuracy: 0.50, Eval Accuracy: 0.60
Test Accuracy: 0.28
Epoch 600, Loss: 22.7091, Accuracy: 0.64, Eval Accuracy: 0.55
Test Accuracy: 0.28
Epoch 700, Loss: 44.9908, Accuracy: 0.53, Eval Accuracy: 0.55
Test Accuracy: 0.28
Epoch 800, Loss: 38.2070, Accuracy: 0.53, Eval Accuracy: 0.55
Test Accuracy: 0.28
Epoch 900, Loss: 43.9573, Accuracy: 0.56, Eval Accuracy: 0.45


# Evalution

In [34]:
logits = np.dot(X_test, W) + dw0
probs = softmax(logits)
preds = np.argmax(probs, axis=1)
accuracy = accuracy_score(y_test, preds)
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.40


# Inferrence

In [40]:
# Inter Milan
homeTeamName = "Inter Milan"
homeAveragePossesion = 56.5
homeAverageShots = 14.6
homeAverageShotsOnTarget = 4.9
homeAverageFouls = 10.8
homeAverageSaves = 5

# PSG
awayTeamName = "PSG"
awayAveragePossesion = 66.6
awayAverageShots = 18.8
awayAverageShotsOnTarget = 7.9
awayAverageFouls = 10
awayAverageSaves = 3

result_map = {
    0: "Draw",
    1: f"{homeTeamName} Win",
    2: f"{awayTeamName} Win"
}

X = np.array([[homeAveragePossesion, awayAveragePossesion, homeAverageShotsOnTarget, awayAverageShotsOnTarget, homeAverageShots, awayAverageShots, homeAverageFouls, awayAverageFouls, homeAverageSaves, awayAverageSaves]])
logits = np.dot(X, W) + W0
probs = softmax(logits)
preds = np.argmax(probs, axis=1)
print(f"Predicted class: {preds[0]}")
print(f"Predicted result: {result_map[preds[0]]}")
print(f"Predicted probabilities: {[f'{p:.2f}' for p in probs[0]]}")
print(f'Model Accuracy: {accuracy:.2f}')

 

Predicted class: 2
Predicted result: PSG Win
Predicted probabilities: ['0.00', '0.00', '1.00']
Model Accuracy: 0.40


In [37]:
W

array([[-2.09372238,  1.88773327, -0.15795893],
       [-1.31428184, -0.69160968,  1.46806215],
       [-4.11107255, 13.09999116, -9.58185264],
       [-5.87025662, -1.81247039,  7.34305528],
       [ 0.44444013,  1.55931885, -1.70542877],
       [-0.17314797, -6.18155102, 10.11434286],
       [-1.53920681,  3.21825804, -1.16501836],
       [-0.85678204, -1.18274735, -0.89966246],
       [ 5.60636955,  5.93442395, -9.54482793],
       [ 1.1862731 , -6.19075366,  2.46347299]])